In [1]:
import pandas as pd
import numpy as np
import quandl

In [8]:
df= pd.read_csv("EOD-AAPL.csv")

In [9]:
df.head()

,Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
0,2017-12-28,171.00,171.850,170.480,171.08,16480187.0,0.0,1.0,41.136972,41.341454,41.011877,41.156217,65920748.0
1,2017-12-27,170.10,170.780,169.710,170.60,21498213.0,0.0,1.0,40.920461,41.084047,40.826640,41.040745,85992852.0
2,2017-12-26,170.80,171.470,169.679,170.57,33185536.0,0.0,1.0,41.088858,41.250038,40.819183,41.033528,132742144.0
3,2017-12-22,174.68,175.424,174.500,175.01,16349444.0,0.0,1.0,42.022259,42.201241,41.978957,42.101646,65397776.0
4,2017-12-21,174.17,176.020,174.100,175.01,20949896.0,0.0,1.0,41.899570,42.344619,41.882730,42.101646,83799584.0


In [10]:
df.shape

(1090, 13)

In [11]:
df["HL_PCT"]=(df["Adj_High"]-df["Adj_Low"])/df["Adj_Close"]*100

In [12]:
df["PCT_change"]=(df["Adj_Close"]-df["Adj_Open"])/df["Adj_Open"]*100

In [13]:
df_future= df.loc[:,['Adj_Close','Volume',"PCT_change","HL_PCT"]]

In [14]:
df_future.head()

,Adj_Close,Volume,PCT_change,HL_PCT
0,41.156217,16480187.0,0.046784,0.800795
1,41.040745,21498213.0,0.293945,0.627198
2,41.033528,33185536.0,-0.134660,1.050009
3,42.101646,16349444.0,0.188917,0.527970
4,42.101646,20949896.0,0.482287,1.097080


In [17]:
df_future.fillna(value=-99999, inplace=True)

In [19]:
import math
forecast_out = int(math.ceil(0.01 * len(df_future)))

In [21]:
df_future['label'] = df_future["Adj_Close"].shift(-forecast_out)

In [23]:
df_future

,Adj_Close,Volume,PCT_change,HL_PCT,label
0,41.156217,16480187.0,0.046784,0.800795,41.305369
1,41.040745,21498213.0,0.293945,0.627198,41.538719
2,41.033528,33185536.0,-0.134660,1.050009,40.744848
3,42.101646,16349444.0,0.188917,0.527970,40.732819
4,42.101646,20949896.0,0.482287,1.097080,40.658243
...,...,...,...,...,...
1085,16.033661,12167400.0,0.231683,0.877176,NaN
1086,15.781833,12840200.0,-0.044138,1.892738,NaN
1087,15.688388,8441700.0,-0.995502,1.421447,NaN
1088,15.796753,12322600.0,-0.173953,1.195129,NaN


In [24]:
df_future=df_future.dropna()

In [26]:
df_future

,Adj_Close,Volume,PCT_change,HL_PCT,label
0,41.156217,16480187.0,0.046784,0.800795,41.305369
1,41.040745,21498213.0,0.293945,0.627198,41.538719
2,41.033528,33185536.0,-0.134660,1.050009,40.744848
3,42.101646,16349444.0,0.188917,0.527970,40.732819
4,42.101646,20949896.0,0.482287,1.097080,40.658243
...,...,...,...,...,...
1074,15.492944,13012300.0,-1.167960,1.564097,16.033661
1075,15.541726,27218100.0,-1.100585,2.916599,15.781833
1076,14.805882,24975100.0,-2.215481,2.685009,15.688388
1077,14.960780,14447900.0,0.339919,1.393182,15.796753


In [56]:
X = df_future.drop(['label'],axis=1)

In [57]:
y=df_future["label"]

In [58]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X= sc.fit_transform(X.values)
y=sc.fit_transform(y.values.reshape(-1,1))


In [61]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [63]:
linclf = LinearRegression()
linclf.fit(X_train, y_train)

LinearRegression()

In [64]:
# Predicting the Test set results
y_pred = linclf.predict(X_test)

In [65]:
from sklearn.metrics import mean_squared_error, r2_score
# The mean squared error
print("Mean squared error: {}".format(mean_squared_error(y_test, y_pred)))
# Explained variance score: 1 is perfect prediction
print("Variance score: {}".format(r2_score(y_test, y_pred)))

Mean squared error: 0.02315189686646055
Variance score: 0.9754519381796016
